In [1]:
#Importing needed packages
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [2]:
# Load the raw data from excel
nordpool_price = pd.read_csv('Data/NordPool.csv', delimiter=';')
energinet_price = pd.read_csv('Data/Energinet Balance price.csv', delimiter=';')
production_forecast = pd.read_csv('Data/Wind Production Forecast.csv')

# Filter the Energinet data to only include zone DK2
filtered_energinet_price = energinet_price[energinet_price['PriceArea | PriceArea | 804696'] == 'DK2']
energinet_price = filtered_energinet_price

# Convert the 'ts' column to datetime format for each dataset
nordpool_price['ts'] = pd.to_datetime(nordpool_price['ts'], dayfirst=True)
energinet_price['ts'] = pd.to_datetime(energinet_price['ts'], dayfirst=True)

# Set 'ts' as the index for each DataFrame
nordpool_price.set_index('ts', inplace=True)
energinet_price.set_index('ts', inplace=True)

#Filter out relevant columns for balancing prices list
columns_to_keep = ['BalancingPowerPriceUpEUR | BalancingPowerPriceUpEUR | 804718', 'BalancingPowerPriceDownEUR | BalancingPowerPriceDownEUR | 804720'] 
energinet_price = energinet_price[columns_to_keep]




/var/folders/j_/mwl680ps3b1czvrppsb_jxgm0000gn/T/ipykernel_58853/4136654533.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energinet_price['hour'] = energinet_price.index.hour


In [3]:
#Create an average day for day ahead and balancing market
nordpool_price['hour'] = nordpool_price.index.hour
average_nordpool = nordpool_price.groupby('hour').mean()

energinet_price['hour'] = energinet_price.index.hour
average_energienet = energinet_price.groupby('hour').mean()

/var/folders/j_/mwl680ps3b1czvrppsb_jxgm0000gn/T/ipykernel_58853/4083495345.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energinet_price['hour'] = energinet_price.index.hour


In [12]:
# Print average_nordpool and average_energienet
print(average_nordpool)
print(average_energienet)

      Nordpool Elspot Prices - hourly price DK-DK2 EUR/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_eur | 3038  \
hour                                                                                                          
0                                             72.312000                                                       
1                                             67.396667                                                       
2                                             67.323667                                                       
3                                             65.062000                                                       
4                                             68.231000                                                       
5                                             72.960000                                                       
6                                             91.893000                                                       
7

In [5]:
Number_of_Hours = 24
NamePlate_Capacity = 100

In [6]:
#Defining the gurobi model

model = gp.Model("Wind Power Optimization")

#Defining the variables

#Variable for the energy that is bid in the day ahead market
DA_Bid = model.addVars(Number_of_Hours, vtype=GRB.CONTINUOUS, name="DA_Bid")

#Variable if the production was actually higher than what was bid on the day ahead market, this number will be sold on the balancing market
Balance_Up = model.addVars(Number_of_Hours, vtype=GRB.CONTINUOUS, name="Balance_Up")

#Variable if the production was actually lower than what was bid on the day ahead market, this deficit will be purchased on the balancing market
Balance_Down = model.addVars(Number_of_Hours, vtype=GRB.CONTINUOUS, name="Balance_Down")

#Variable that describes the delta between day ahead bid and actual production
Delta = model.addVars(Number_of_Hours, vtype=GRB.CONTINUOUS,lb=-GRB.INFINITY, name="Delta")

#Defining objective function
objective = (
    gp.quicksum(average_nordpool.loc[h, 'Nordpool Elspot Prices - hourly price DK-DK2 EUR/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_eur | 3038'] * DA_Bid[h] for h in range(Number_of_Hours))
    + gp.quicksum(average_energienet.loc[h, 'BalancingPowerPriceDownEUR | BalancingPowerPriceDownEUR | 804720'] * Balance_Up[h] for h in range(Number_of_Hours))
    - gp.quicksum(average_energienet.loc[h, 'BalancingPowerPriceUpEUR | BalancingPowerPriceUpEUR | 804718'] * Balance_Down[h] for h in range(Number_of_Hours))
)
model.setObjective(objective, GRB.MAXIMIZE)


for t in range(Number_of_Hours):
    #Adding Constraint that defines delta as the difference between the day ahead bid and the actual power production. Delta is positive for overproduction, negative for underproduction
    model.addConstr(Delta[t] == production_forecast.loc[t,'Wind Production [MW]'] - DA_Bid[t], name=f"Defining Delta_{t}")
    #Adding constraint that defines the value for balance_up/down
    model.addConstr(Delta[t] == Balance_Up[t] - Balance_Down[t] , name=f"Defining Delta Up/Down_{t}")
    #Constraint that makes sure, that bid can´t be higher than the maximum capacity of the wind farm
    model.addConstr( DA_Bid[t] <= NamePlate_Capacity, name=f"Production Capacity_{t}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-24


In [11]:
model.optimize()


# # Check optimization result
if model.status == GRB.INFEASIBLE:
    print("Model is infeasible.")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
elif model.status == GRB.TIME_LIMIT:
    print("Time limit reached.")
elif model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    TotalRevenue = model.objVal
    print("Total Revenue: ", TotalRevenue)
    #Save results in data frame

    results = {
        'DA_Bid': [DA_Bid[t].X for t in range(Number_of_Hours)],
        'Wind_Production_forecast': [production_forecast.loc[t,'Wind Production [MW]'] for t in range(Number_of_Hours)],
        'Balance_Up': [Balance_Up[t].X for t in range(Number_of_Hours)],
        'Balance_Down': [Balance_Down[t].X for t in range(Number_of_Hours)],
        'Delta': [Delta[t].X for t in range(Number_of_Hours)]
    }

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 72 rows, 96 columns and 144 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 1e+02]

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.385503456e+05
Optimal solution found.
Total Revenue:  138550.3455998172


In [10]:
# Create a DataFrame from the dictionary
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)


    DA_Bid  Wind_Production  Balance_Up  Balance_Down  Delta
0    100.0              100         0.0           0.0    0.0
1     56.0               56         0.0           0.0    0.0
2     34.0               34         0.0           0.0    0.0
3     87.0               87         0.0           0.0    0.0
4     45.0               45         0.0           0.0    0.0
5      0.0               86        86.0           0.0   86.0
6      0.0               45        45.0           0.0   45.0
7      0.0               85        85.0           0.0   85.0
8     23.0               23         0.0           0.0    0.0
9     86.0               86         0.0           0.0    0.0
10    54.0               54         0.0           0.0    0.0
11    22.0               22         0.0           0.0    0.0
12    75.0               75         0.0           0.0    0.0
13    93.0               93         0.0           0.0    0.0
14    44.0               44         0.0           0.0    0.0
15    64.0              